ここで、NISQデバイスの応用先の一つと考えられている、量子乱数生成について紹介する（難易度：高）。

# コラム：量子乱数生成

NISQアルゴリズムは近未来の量子技術で実現可能な応用として注目されている。一方、計算機に限らず一般的な情報処理までスコープを広げれば、より基本的な技術で実現可能な量子情報処理の応用がいくつか考えられる。特に「量子乱数生成」「量子センシング」「量子暗号」の三つは実現がNISQより容易な応用としてしばしば名前があがる。このコラムではこうした応用の一つである、量子乱数に関する研究について簡単に解説する。

乱数の応用はモンテカルロ法を用いた計算や暗号における乱数列の生成など多岐にわたる。こうした応用において、乱数に要請される性質は0,1の出現に関してバイアスがないこと、そして以前に生成した乱数との相関がないことである。こうした理想的な性質を持つ乱数を真正乱数と呼ぶ。しかし、厳密な真正乱数の生成は現実には困難であるため、通常の計算機においては現実的なコストで実現可能な物理乱数や疑似乱数が用いられる。

物理乱数とは一般に予測が困難な計算機の物理的な情報をモニタリングし、その情報からバイアスや相関を取り去ることで生成する乱数である。例えばCPUや通信機器のノイズ情報が用いられる。こうして得られた乱数はエントロピープールと呼ばれる箇所に蓄積され、物理乱数をプログラムが使用するたびに消費される。
物理乱数の利点は予測が困難なことである。物理乱数の欠点は、生成がデバイスのノイズに依存するため概して生成が遅いことと、予測は困難であっても理論上不可能ではないことである。例えばCPUの熱ノイズを遠くにいる攻撃者が正確に予測することは現実的には困難だが、攻撃者が計算機の近くにいる場合、計算機の本体から漏れ出る情報をモニタリングすることで、計算機の使用者が想定した以上の情報が外に漏れ出てしまうという可能性はある。また、さらに理論上の話をすれば、古典力学的に扱える物理系は初期状態とダイナミクスが定まれば理論上は完全に予測可能である。

もう一つの乱数は疑似乱数である。疑似乱数とは、ある秘密の「状態」を指定し、これを決定論的に遷移させることで、遷移に際して現在の「状態」を知らない人にとって乱数と区別がつかないと期待される数列を得ることである。例として、線形合同法、xorshift、メルセンヌツイスタのようなアルゴリズムが代表的である。
疑似乱数の特徴は物理乱数と相補的であり、利点は高速であること、欠点は現在の「状態」が外部に漏れると以降の数列がすべて予測されてしまうことである。このため、暗号鍵生成のような安全性が重視される場面では物理乱数が、物理シミュレーションのような高速性が重視される場面では初期状態として物理乱数を用いて以降は疑似乱数を用いることが多い。

量子乱数は物理系の持つ予測の難しさを利用するという意味では物理乱数の一種であるが、初期状態を攻撃者が把握していても原理的に未来に生成される乱数が予測が不可能であるという点で熱ノイズなどを用いた乱数とは異なる。量子乱数の最も基本的なアイデアは、量子状態を$|+\rangle$に初期化し、これを$Z$基底で測定するものである。

In [ ]:
## Google Colaboratoryの場合・Qulacsがインストールされていないlocal環境の場合のみ実行してください
!pip install qulacs

In [1]:
## qulacs (3-1節)というパッケージを使用します
from qulacs import QuantumState
from qulacs.gate import H
state = QuantumState(1)
state.set_zero_state()
gate = H(0)
gate.update_quantum_state(state)

count = 10000
result = np.array(state.sampling(count))
print("0: {}".format(np.sum(result==0)))
print("1: {}".format(np.sum(result==1)))


0: 4938
1: 5062


上記のコードと全く同じ操作を現実に行えば毎回50%の確率で0,1が出現する。（上記はシミュレータなので実際には物理乱数をシードにした疑似乱数である。）

$|+\rangle$を用意して$Z$基底で測定する操作は一見すると熱ノイズなどを扱う物理乱数とあまり変わらないように思えるが、乱数生成の背景を攻撃者が完全に知っていたとしてもどういった値が出るのか理論上予測できないという点で物理乱数とは大きく異なる。完ぺきに調整された信頼できる量子乱数源は、たとえ攻撃者がその初期状態を知っていたとしても真正乱数となる。

量子乱数生成の研究の目的は、上記のような理想的な状況をより現実に近い制約の下で実現することである。
量子乱数生成に関する研究の方向性は多岐にわたるが、このコラムでは代表的な二つの応用に向けた方向性について簡単に説明する。

## 安価な実験で量子乱数を実現する

イオンや超伝導素子は理想的な量子ビットを生成するためには良い物質だが、その準備には光源/マイクロ波源や冷却機構などの高度な技術と高価な装置が必要となる。
しかし現実的な問題として、乱数を生成するためだけに希釈冷凍機や真空チャンバを購入することは殆どの場合割に合わない。
このため、いかに安価な実験装置でも量子乱数を生成できるかというテーマは応用上重要である。
以下に一見すると単なる物理乱数に見えるが、実は初期状態を攻撃者が完全に把握していたとしても予測が出来ない量子乱数となっている例として以下の２つがある。

### 光のショットノイズ

発振したレーザーから放出される微弱な光を高精度な光強度検出器で観測すると、ショットノイズと呼ばれるノイズが不可避に乗ることが知られている。
これは、発振したレーザーから放出されるパルスは光子数分布がポアソン分布と整合するコヒーレント状態と呼ばれる純粋状態となるために、その光子数の期待値は一定であっても、光子数の射影測定によって得られる光子の数がばらついてしまうことによる。このばらつきのことをショットノイズと呼ぶ。光の強度に対するショットノイズの大きさは光の強度が小さくなればなるほど大きくなる。

通常、こうしたばらつきは精密な撮像などでSN比向上のために排除するべきノイズとなる。一方、量子乱数の文脈では純粋状態に対する射影測定の結果生じる確率的な挙動であるから量子乱数として用いることができる。もちろん、検出される光子数は0,1で等確率ではないし2光子が検出されることもあるため、適切な補正が必要となる。
また、当然ながら測定器やレーザーの電流値揺らぎによるノイズは量子乱数とはみなせず、現実的なレーザー光の光子数には時間的な相関も存在する。このため、実際にこのシステムで量子乱数を構成するにはショットノイズ以外の古典的ノイズの要因が無視できる程度にシステムが安定している必要はある。

### 原子核のアルファ崩壊

原子核のアルファ粒子は原子核のポテンシャルに閉じ込められているが、トンネル効果のために一定の確率でポテンシャルを通り抜けて原子核外に放出される。トンネル効果によって波動関数がポテンシャル内側と外側に分かれるという操作はユニタリな操作であるため、これは透過率が小さなミラーに光子を当て、透過したか否かを射影測定することと現象としては等価である。 
従って、適切なレートでアルファ粒子を放出する原子集団を準備し、検知器の検知タイミングに適切な補正を施すことで量子乱数源とすることができる。
なお原子核のこうしたふるまいのエネルギースケールは室温ノイズのエネルギースケールよりはるかに大きいため、上記の観測は室温でも行える。


## 信頼できない量子デバイスで量子乱数を生成する

もしチップのベンダに悪意があり、CPUの熱ノイズを取得する関数が、メルセンヌツイスタに従う疑似乱数を生成していたとしても、それを我々が検定することは困難である。従って、CPUの熱ノイズを物理乱数として用いるとき、自身がCPUの中身を検査する能力を持たない限りはCPUのベンダが用意した命令の挙動を暗黙に信頼していることになる。現実にはCPUやチップのこうした挙動は最終的には会社の信頼度によって担保されていると思われる。

同様に、もし「量子乱数生成装置」なるものが販売されていたとして、その中身がブラックボックスとなっているとき、この装置を使うことは量子乱数生成のベンダを信頼していることを前提とする。
量子乱数生成器のベンダが社会的信頼をまだ勝ち得ていない場合、デバイスの信頼性の欠如は物理乱数を利用すること以上のリスクとなるため、販売における大きな障害となる。

保障あり量子乱数生成(Certified Quantum Random Number Generation)の研究とは、何らかの現実的な仮定をおくことで信頼できないベンダによる量子乱数生成装置から信頼できる量子乱数を生成する手法を模索する分野である。この中で最も有名なものは、量子操作を行える離れた二つの信頼できないサーバ、信頼できる古典計算機、シードとなる少量の乱数を用いて、信頼できる量子乱数を生成する手法である[\[S. Pironio, et al., Nature 464, 1021 (2010)\]](https://arxiv.org/abs/0911.3427)。この手法ではベル測定の破れを二つの敵対的な量子計算サーバによって検証することで、保証された量子乱数を得るといったものである。ベル不等式の上界に古典と量子で差が出る理由は、ベル不等式の量子演算子に対し整合的なjoint probaility distributionを定義できないことに依存する。従って、ベル不等式が破れている状況では、その測定結果に不可避なランダムネスが生じているということである。下記の手法はベル不等式におけるこの性質を利用している。

手順はかみ砕くと以下のようなものである。

二つの離れたサーバをA,B、乱数を生成するプレイヤーをPとする。この時、登場人物はA,P,Bの順番に一次元に並んでおり、それぞれ$L$だけ距離が離れているとする。光速を$c$とする。この時、以下の手順を行う。

1. 二つの離れたサーバA,Bの間で、ベル状態を共有させる。
2. Pは$O(\sqrt{n})$ビットの乱数をエントロピープールから消費し、所定の手順でこの乱数から$2n$ビットの数列を作る。
3. Pは$2n$ビットのうち$n$ビットをAに、$n$ビットをBに送付する。
4. A,Bは指定されたビットに従った基底で事前に共有したベル状態の片割れを測定し、その結果をPに返却する。
5. Pは、3.の送信から4.の返却までの手続きに$4L/c$秒以上かかっていた場合、プロトコルを破棄し1.からやり直す。
6. PはAとBが誠実に測定したと仮定し、演算子の期待値からベル不等式の破れを検証する。ベル不等式を破っている度合いに応じて受け取った$2n$ビットを縮小し、その結果を乱数とする。

上記のプロトコルではA,Bが誠実である場合、Pは$O(n)$ビットを得る。
A,Bが不誠実で会った場合、例えば与えられた指示を無視した基底でベル状態を測定したり、Pに隠れてA,B間で通信を行って共謀してPをだまそうとする。
しかし、A,B間の通信に上記の時間制約があり、かつベル不等式が確かに破れている場合、上記のようなズルではPが乱数を得ることは妨害できない。
従って、「Pは乱数だと思って乱数を続けるが、実はその乱数はAやBによって操作された数列である」という可能性をなくすことができる。

もちろん、返却までに時間がかかったり、ベル不等式を破らない返答をすることは可能である。この場合PはA,Bが不審であると考え、乱数生成のプロトコル自体を破棄することになる。あくまで「Pが乱数だと思っているものが、実は乱数ではなかった」というケースを排除するプロトコルであって、悪意あるサーバA,Bの妨害を防ぐことができるものではない。

また、このプロトコルを開始するには$O(\sqrt{n})$の量子乱数が開始に必要となる。このため、このプロトコルは純粋な意味での量子乱数生成ではなく、正しくは量子乱数増幅である点にも注意が必要である。